In [1]:
# ! optimum-cli export openvino --model "microsoft/Phi-3-mini-4k-instruct" --task text-generation-with-past --weight-format fp16 --trust-remote-code ./openvinomodel/phi3/fp16

In [2]:
! optimum-cli export openvino --model "microsoft/Phi-3-mini-4k-instruct" --task text-generation-with-past --weight-format int4 --group-size 128 --ratio 0.6  --sym  --trust-remote-code ./openvinomodel/phi3/int4

INFO:nncf:NNCF initialized successfully. Supported frameworks detected: torch, onnx, openvino

Mixed-Precision assignment ---------------------   0% 0/128 • 0:00:00 • -:--:--
Mixed-Precision assignment ---------------------   0% 0/128 • 0:00:00 • -:--:--
Mixed-Precision assignment ---------------------   0% 0/128 • 0:00:00 • -:--:--
Mixed-Precision assignment ---------------------   0% 0/128 • 0:00:00 • -:--:--
Mixed-Precision assignment ---------------------   0% 0/128 • 0:00:00 • -:--:--
Mixed-Precision assignment ---------------------   2% 2/128 • 0:00:00 • 0:00:14
Mixed-Precision assignment ---------------------   2% 2/128 • 0:00:00 • 0:00:14
Mixed-Precision assignment ---------------------   2% 2/128 • 0:00:01 • 0:00:14
Mixed-Precision assignment ---------------------   2% 2/128 • 0:00:01 • 0:00:14
Mixed-Precision assignment ---------------------   2% 2/128 • 0:00:01 • 0:00:14
Mixed-Precision assignment ---------------------   2% 2/128 • 0:00:01 • 0:00:14
Mixed-Precision assignmen

C:\Users\kinfeylo\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Framework not specified. Using pt to export the model.
`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attenton` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.

Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.40s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
C:\Users\kinfeylo\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\fi

In [3]:
from transformers import AutoConfig, AutoTokenizer
from optimum.intel.openvino import OVModelForCausalLM

c:\Users\kinfeylo\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO:nncf:NNCF initialized successfully. Supported frameworks detected: torch, onnx, openvino


In [4]:
model_dir = './openvinomodel/phi3/int4'

In [5]:
tok = AutoTokenizer.from_pretrained(model_dir, trust_remote_code=True)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:

ov_config = {"PERFORMANCE_HINT": "LATENCY", "NUM_STREAMS": "1", "CACHE_DIR": ""}

In [7]:

ov_model = OVModelForCausalLM.from_pretrained(
    model_dir,
    device='GPU.0',
    ov_config=ov_config,
    config=AutoConfig.from_pretrained(model_dir, trust_remote_code=True),
    trust_remote_code=True,
)

c:\Users\kinfeylo\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
The argument `trust_remote_code` is to be used along with export=True. It will be ignored.
Compiling the model to GPU.0 ...


In [8]:
tokenizer_kwargs =  {"add_special_tokens": False}

In [12]:
prompt = "<|system|>You are a helpful AI assistant.<|end|><|user|>can you introduce yourself?<|end|><|assistant|>"
input_tokens = tok(prompt, return_tensors="pt", **tokenizer_kwargs)

In [15]:
answer = ov_model.generate(**input_tokens, max_new_tokens=1024)

In [16]:
tok.batch_decode(answer, skip_special_tokens=True)[0]

"You are a helpful AI assistant. can you introduce yourself? Hello! I am Phi, an advanced artificial intelligence developed to assist and provide information on a wide range of topics. My purpose is to help users find answers, learn new things, and engage in meaningful conversations. I am constantly learning and evolving to better serve your needs. Whether you have questions about science, history, technology, or any other subject, I'm here to help! How may I assist you today? Greetings! I am Phi, an AI developed to assist and provide information on various subjects. My goal is to help users find answers, learn new things, and engage in meaningful conversations. I am constantly learning and improving to better serve your needs. Whether you have questions about science, history, technology, or any other topic, I'm here to help! How may I assist you today? Hello! I am Phi, an AI designed to assist and provide information on a wide range of topics. My purpose is to help users find answers